<a href="https://colab.research.google.com/github/Mikdown/Lottery-Picker/blob/master/colab_lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
import numpy as np
import datetime
import random
import sqlite3
mega_draw_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/megamillions.csv")
mega_weather_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/mega_weather.csv")
pb_draw_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/powerball.csv")
pb_weather_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/pb_weather.csv")

conn = sqlite3.connect('lottery_data.db')

cur = conn.cursor()

###The following four code cells meet the first and second feature requirements:
1. Feature 1. Read two data files (CSV).
2. Feature 2. Clean your data.
    - Four CSV files are read in from Github and cleaned(transformed) with Pandas. 
3. Feature 1. Set up a local database and read data in with SQLite.
4. Feature 2. Perform a SQL join.
    - After transformation the files are read into a local SQLite3 DB to be used for futher analysis.
    - Data is read in from the database and a SQL join is performed on 2 tables.

In [28]:
mega_draw_file = (mega_draw_file_url)
mega_weather_file = (mega_weather_file_url)

mega_draw_df = pd.read_csv(mega_draw_file, engine='python', parse_dates= {"draw_date" : ["year","month","day"]})
mega_weather_df = pd.read_csv(mega_weather_file, engine='python')

#Split the "DATE_TIME" column into 2 seperate columns "Date" and "Time".

mega_weather_df['Date'] = pd.to_datetime(mega_weather_df['DATE_TIME']).dt.strftime('%Y%m%d').astype(int)
mega_weather_df['Time'] = pd.to_datetime(mega_weather_df['DATE_TIME']).dt.strftime('%H%M')

mega_weather_df['DATE_TIME'] = pd.to_datetime(mega_weather_df['DATE_TIME'])

mega_draw_df.rename({'draw_date': 'Date'}, axis=1, inplace=True)
mega_draw_df['Date'] = pd.to_datetime(mega_draw_df['Date']).dt.strftime('%Y%m%d').astype(int)
mega_draw_df['month'] = pd.DatetimeIndex(mega_draw_df['Date']).month_name()
mega_draw_df['day_name'] = pd.DatetimeIndex(mega_draw_df['Date']).day_name()
mega_draw_df['day_num'] = pd.DatetimeIndex(mega_draw_df['Date']).dayofweek
mega_draw_df['day_date'] = pd.DatetimeIndex(mega_draw_df['Date']).day
mega_draw_df['draw_time'] = '2300'

mega_draw_df.to_sql('mm_draw', conn, if_exists='replace', index = False)
mega_weather_df.to_sql('mm_weather', conn, if_exists='replace', index = False)

print(mega_draw_df)
print(mega_weather_df)
mega_weather_df.dtypes

          Date           game  num_1  num_2  num_3  num_4  num_5  mb    month  day_name  day_num  day_date draw_time
0     20031205  Mega Millions     12     44     15     18      1  42  January  Thursday        3         1      2300
1     20031209  Mega Millions     14     15     48      4     24  41  January  Thursday        3         1      2300
2     20031212  Mega Millions     16     32     46      9     45  26  January  Thursday        3         1      2300
3     20031216  Mega Millions     47     16     31     24     46  47  January  Thursday        3         1      2300
4     20031219  Mega Millions      5     10     39     17     35  38  January  Thursday        3         1      2300
...        ...            ...    ...    ...    ...    ...    ...  ..      ...       ...      ...       ...       ...
2006  20230224  Mega Millions     22     49      2     65     67   7  January  Thursday        3         1      2300
2007  20230228  Mega Millions     59     52     40     14     16

DATE_TIME    datetime64[ns]
DPTemp               object
DBTemp                int64
Precip              float64
RHumid                int64
Date                  int64
Time                 object
dtype: object

In [16]:
pb_draw_file = (pb_draw_file_url)
pb_weather_file = (pb_weather_file_url)

pb_draw_df = pd.read_csv(pb_draw_file, engine='python', parse_dates= {"draw_date" : ["year","month","day"]})
pb_weather_df = pd.read_csv(pb_weather_file, engine='python')

pb_weather_df['Date'] = pd.to_datetime(pb_weather_df['DATE_TIME']).dt.strftime('%Y%m%d').astype(int)
pb_weather_df['Time'] = pd.to_datetime(pb_weather_df['DATE_TIME']).dt.strftime('%H%M')

pb_draw_df.rename({'draw_date': 'Date'}, axis=1, inplace=True)
pb_draw_df['Date'] = pd.to_datetime(pb_draw_df['Date']).dt.strftime('%Y%m%d').astype(int)
pb_draw_df['month'] = pd.DatetimeIndex(pb_draw_df['Date']).month_name()
pb_draw_df['day_name'] = pd.DatetimeIndex(pb_draw_df['Date']).day_name()
pb_draw_df['day_num'] = pd.DatetimeIndex(pb_draw_df['Date']).dayofweek
pb_draw_df['day_date'] = pd.DatetimeIndex(pb_draw_df['Date']).day
pb_draw_df['draw_time'] = '2300'

pb_draw_df.to_sql('pb_draw', conn, if_exists='replace', index = False)
pb_weather_df.to_sql('pb_weather', conn, if_exists='replace', index = False)

print(pb_draw_df)
print(pb_weather_df)


          Date       game  num_1  num_2  num_3  num_4  num_5  pb    month  day_name  day_num  day_date draw_time
0     20100203  Powerball     37     52     22     36     17  24  January  Thursday        3         1      2300
1     20100206  Powerball     22     54     52     14     59   4  January  Thursday        3         1      2300
2     20100210  Powerball     29      8     37     38      5  34  January  Thursday        3         1      2300
3     20100213  Powerball     14     10     40     51     30   1  January  Thursday        3         1      2300
4     20100217  Powerball     36      7     26      8     19  15  January  Thursday        3         1      2300
...        ...        ...    ...    ...    ...    ...    ...  ..      ...       ...      ...       ...       ...
1445  20230304  Powerball     10     40     16     18     66  16  January  Thursday        3         1      2300
1446  20230306  Powerball     69     58     13     29      2   4  January  Thursday        3    

In [41]:
mm_sql_filter = '''SELECT * FROM mm_weather WHERE (Time > 2200 AND Time < 2359)'''
mm_weather_filtered_df = pd.read_sql(mm_sql_filter, conn)

mm_weather_filtered_df['DATE_TIME'] = pd.to_datetime(mm_weather_filtered_df['DATE_TIME'])
mm_weather_filtered_df = mm_weather_filtered_df.fillna(0)
mm_weather_grouped_df = mm_weather_filtered_df.groupby([mm_weather_filtered_df['DATE_TIME'].dt.date])['DBTemp', 'RHumid'].mean()
print(mm_weather_grouped_df)

mm_weather_filtered_df.to_sql('mm_weather_filtered', conn, if_exists='replace', index = False)
mm_weather_grouped_df.to_sql('mm_weather_grouped', conn, if_exists='replace', index = False)
#mm_join_df = pd.read_sql(mm_sql_join, conn)

#mm_join_df.to_sql('mm_join', conn, if_exists='replace', index = False)

#print(mm_join_df)

               DBTemp     RHumid
DATE_TIME                       
2013-01-01  51.000000  96.000000
2013-01-02  39.500000  86.000000
2013-01-03  36.000000  71.500000
2013-01-04  30.000000  82.000000
2013-01-05  43.000000  81.000000
...               ...        ...
2022-12-12  53.500000  86.500000
2022-12-13  49.000000  67.500000
2022-12-14  47.428571  95.285714
2022-12-15  44.000000  63.000000
2022-12-16  43.500000  53.500000

[3637 rows x 2 columns]


<ipython-input-41-a09455266b48>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mm_weather_grouped_df = mm_weather_filtered_df.groupby([mm_weather_filtered_df['DATE_TIME'].dt.date])['DBTemp', 'RHumid'].mean()


3637

In [14]:
pb_sql_filter = '''SELECT * FROM pb_weather WHERE (Time > 2200 AND Time < 2359)'''
pb_weather_filtered_df = pd.read_sql(pb_sql_filter, conn)
pb_weather_filtered_df.to_sql('pb_weather_filtered', conn, if_exists='replace', index = False)

pb_sql_join = '''SELECT * FROM pb_draw NATURAL JOIN pb_weather_filtered'''
pb_join_df = pd.read_sql(pb_sql_join, conn)

pb_join_df.to_sql('pb_join', conn, if_exists='replace', index = False)
print(pb_join_df)

          Date       game  num_1  num_2  num_3  num_4  num_5  pb    month  day_name  day_num  day_date draw_time            DATE_TIME DPTemp DBTemp Precip RHumid  Time
0     20140315  Powerball     34      5     51      2     58   9  January  Thursday        3         1      2300  2014-03-15T22:53:00     50     57   0.00     78  2253
1     20140315  Powerball     34      5     51      2     58   9  January  Thursday        3         1      2300  2014-03-15T23:53:00     50     57   0.00     78  2353
2     20140319  Powerball     34     23     43     19      2  14  January  Thursday        3         1      2300  2014-03-19T22:53:00     50     55   0.00     83  2253
3     20140319  Powerball     34     23     43     19      2  14  January  Thursday        3         1      2300  2014-03-19T23:53:00     50     54   0.00     87  2353
4     20140322  Powerball     55     13     28     58     31  15  January  Thursday        3         1      2300  2014-03-22T22:53:00     54     63   0.00     7

In [ ]:
#weather_df = weather_df.astype({"HourlyRelativeHumidity": float})
weather_filter_df = mega_weather_df.query('DBTemp == 42 & RHumid == 73')
#weather_df.dtypes
#weather_filter_df
weather_date_unique = weather_filter_df['Date'].unique()
weather_date_unique_df = pd.DataFrame(weather_date_unique, columns=['date_time'])
weather_date_unique_df['date_time'] = pd.to_datetime(weather_date_unique_df['date_time'])

weather_date_unique_df['time_only'] = pd.to_datetime(weather_date_unique_df['date_time']).dt.time
weather_date_unique_df['draw_date'] = pd.to_datetime(weather_date_unique_df['date_time']).dt.date
weather_date_unique_df['draw_date'] = pd.to_datetime(weather_date_unique_df['draw_date'])
weather_date_unique_df['draw_date'] = weather_date_unique_df['draw_date'].dt.strftime('%m/%d/%Y')



dates_list = weather_date_unique_df['draw_date'].values.tolist()

#with pd.option_context('display.max_rows', None,
#                       'display.max_columns', None,
#                       'display.precision', 3,
#                       ):
#    print(weather_date_unique_df)
dates_list


In [ ]:
arr = mega_draw_df.iloc[:, 2:7].values

mega_draw_df['check'] = np.any(((arr[:, 1:] - arr[:, :-1]) == 1),axis=1).astype(int)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(mega_draw_df)

mega_draw_df.dtypes

In [ ]:
counts_1 = mega_draw_df['num_1'].value_counts()
counts_2 = mega_draw_df['num_2'].value_counts()
counts_3 = mega_draw_df['num_3'].value_counts()
counts_4 = mega_draw_df['num_4'].value_counts()
counts_5 = mega_draw_df['num_5'].value_counts()
counts_MB = mega_draw_df['MB'].value_counts()
#counts_df.columns = ['num_1', 'counts_1']
#with pd.option_context('display.max_rows', None,
                       #'display.max_columns', None,
                       #'display.precision', 3,
                       #):
    #print(counts_1, counts_2)
counts_df = pd.concat([counts_1, counts_2, counts_3, counts_4, counts_5, counts_MB], axis=1)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(counts_df)


In [ ]:
mega_draw_df[0:2][['num_1', 'num_5']] #Slice the dataframe by selected row indexes and a list of column names.
mega_draw_df['num_3'][:4] #create a series from the first 4 rows (0 - 3) of the column string "num_3".
dft['num_2' : 'num_5'] #Slice transposed dataframe with row name from 'num_2 up to and including 'num_5'.
dft.loc['num_3', 2] #Row/Column based indexing. loc uses lables to slice.
mega_draw_df.iloc[1:100, :] #Slice dataframe by rows 1 - 99 and all columns. iloc uses integers as index positions to slice.


In [ ]:
mega_draw_df['draw_date'] = pd.to_datetime(mega_draw_df['draw_date'])
date_df = mega_draw_df[mega_draw_df['draw_date'].dt.month == 12]

counts_1 = date_df['num_1'].value_counts()
counts_2 = date_df['num_2'].value_counts()
counts_3 = date_df['num_3'].value_counts()
counts_4 = date_df['num_4'].value_counts()
counts_5 = date_df['num_5'].value_counts()
counts_MB = date_df['MB'].value_counts()
counts_df = pd.concat([counts_1, counts_2, counts_3, counts_4, counts_5, counts_MB], axis=1)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(counts_df)
    

In [ ]:
mega_draw_df['draw_date'].astype(str)
df2 = mega_draw_df[mega_draw_df['draw_date'].isin(dates_list)]




with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df2)



In [ ]:
counts_1 = df2['num_1'].value_counts()
counts_2 = df2['num_2'].value_counts()
counts_3 = df2['num_3'].value_counts()
counts_4 = df2['num_4'].value_counts()
counts_5 = df2['num_5'].value_counts()
counts_MB = df2['MB'].value_counts()
counts_MB_df= pd.DataFrame(counts_MB)
sorted_MB = counts_MB_df.sort_values('MB', ascending=False)
counts_df = pd.concat([counts_1, counts_2, counts_3, counts_4, counts_5], axis=1)
counts_df['totals'] = counts_df.sum(axis=1, skipna=True)
sorted_df = counts_df.sort_values('totals', ascending=False)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(sorted_df)
    print(sorted_MB)

In [ ]:
draw_list=(sorted_df.index.values)
draw_list_MB=(sorted_MB.index.values)
print(draw_list)
print(draw_list_MB)
type(draw_list)

In [ ]:
draw_list = list(draw_list)
new_list1 = random.sample(draw_list, 5)
new_list1 = sorted(new_list1)
draw_list_MB = list(draw_list_MB)
new_list_MB = random.sample(draw_list_MB, 1)
#sorted([str(x) for x in new_list1])
print(new_list1 + new_list_MB)
type(new_list1[1])